In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
from tqdm import tqdm
import os

from neuroanalysis import spike

mouse_name = "ts8_SC"
date = "180415"
session_name = "sine_grating"
#ts9_SC_180419_sine_grating

experiment_id = "{mouse}_{date}_{session}".format(mouse=mouse_name, date=date, session=session_name)
_data = os.path.join("data/3GratingSequence/SpikeTrain/", experiment_id)
_dir = os.path.join("data/3GratingSequence/CHART/", experiment_id)
_temp = "data/3GratingSequence/temp"
if not os.path.isdir(_dir):
    os.mkdir(_dir)

# import data

In [4]:
# trim the marker dataframe into desired format.
marker_table = pd.read_csv(_data+'.csv')
trim = marker_table[marker_table.colorname!='black'].copy(deep=True)
#trim = marker_table[marker_table.marker!='gray'].copy(deep=True)
trim.columns = ['time','marker']
trim.index = np.arange(0,len(trim))
# trim

spike_trains, spike_marker = spike.import_spike_train_data(session=session_name, mouse_id=mouse_name, 
                              mat=_data+".mat",data_dir='.',
                              csv=trim)
list(spike_trains.keys())

marker shift: 0


['WBC02a',
 'WBC02b',
 'WBC03a',
 'WBC04a',
 'WBC06a',
 'WBC08a',
 'WBC09a',
 'WBC10a',
 'WBC11a',
 'WBC12a',
 'WBC14a',
 'WBC15a',
 'WBC16a']

# Preview: spike rasters

In [ ]:
plt.figure(figsize=(320,4))
idx = 0
for (channel, each_train) in spike_trains.items():
    train = each_train.spike_train
    plt.vlines(train, idx-0.5, idx+0.5)
    idx += 1
plt.vlines(spike_marker._raw_train, -2, -1, color='r')
#plt.vlines(spike_marker._raw_table.time[1:-1], -3 ,-2, color='g')
plt.yticks(range(len(spike_trains)),list(spike_trains.keys()))
plt.xlim((0,np.max([i.spike_train[-1] for i in spike_trains.values()])))
#plt.savefig(os.path.join(_dir, experiment_id+"_preivew.png"),bbox_inches='tight')
plt.show()

# seperate ABABA and ABABB trial markers

In [ ]:
idx = 0
ABABB_start = []
ABABA_start = []
while idx < len(spike_marker.table_marker)/5:
    trial = spike_marker.table_marker[idx*5:idx*5+5]
    idx += 1
    #print(idx)
    if trial.marker.values[-1] == 'B':
        ABABB_start.append(trial.time.values[0])
    else:
        ABABA_start.append(trial.time.values[0])
len(ABABB_start)

# calculate mean response for each trial in each channel

with ROI as (-1, 16)

using guassian kernel, with $\sigma=0.4$

save everything in a 2darray into a mat file: "{session\_name}\_{mouse\_name}\_{date}\_{channel}.mat".

In [ ]:
roi = (-2,24)
# 1.5 high, 1.5 low, 
kernel = spike.kernel('gaussian',sigma=0.4)
step = 0.01 # second

nbins = 1000

In [ ]:
# loading old data
_file = os.path.join(_dir, "linear_filter_{setup}.mat")


ABABA_mean_result = scipy.io.loadmat(_file.format(setup='ABABA'))
ABABB_mean_result = scipy.io.loadmat(_file.format(setup='ABABB'))

In [ ]:
ABABA_mean_result = {}
ABABB_mean_result = {}

mat_path = os.path.join(_temp, "{channnel}.mat")
#pbar = tqdm(total=len(spike_trains)*(len(ABABB_start)+len(ABABA_start)))

# pbar = tqdm(total=len())
for (channel, spike_train) in spike_trains.items():
    kk = spike.FilterKernel._gaussian_kernel(0.4, spike_train.spike_train)
    ABABA_mean_result[channel] = spike.FilterKernel._apply_linear_filter_withroi(kk, ABABA_start, 
                                                                                 roi=roi, nbins=nbins)
    ABABB_mean_result[channel] = spike.FilterKernel._apply_linear_filter_withroi(kk, ABABB_start, 
                                                                                 roi=roi, nbins=nbins)
    break

In [6]:
kd = spike.FilterKernel._gaussian_kernel(0.4, spike_trains["WBC11a"].spike_train)
kd(np.linspace(0,10,100))

TypeError: return arrays must be of ArrayType

In [ ]:
def _apply_linear_filter_withroi(k, starts, roi=(0,0), nbins=1000, pbar=None):
    _X = np.linspace(roi[0], roi[1], nbins)
    _result = np.zeros((nbins, len(starts)))
    for idx, each in enumerate(starts):
        X = each + _X
        _result[:,idx] = k(X)
        if pbar:
            pbar.update(1)
    return _result

pbar = tqdm(total=len(ABABA_start))
kd = spike.FilterKernel._gaussian_kernel(0.4, spike_trains["WBC11a"].spike_train)
%time _apply_linear_filter_withroi(kd, ABABA_start, (-2,24), pbar=pbar)

In [ ]:
# processing new data
ABABA_mean_result = {}
ABABB_mean_result = {}

mat_path = os.path.join(_temp, "{channnel}.mat")
pbar = tqdm(total=len(spike_trains)*(len(ABABB_start)+len(ABABA_start)))

for (channel, spike_train) in spike_trains.items():
    mat_name = "{channel}.mat".format(channel=channel)
    mat_path = os.path.join("data/3GratingSequence/temp/", mat_name)
    
    ABABA_mean_result[channel] = spike.apply_linear_filter_withroi(spike_train.spike_train, 
                                                             kernel, ABABA_start, roi=roi, 
                                                             nbins=nbins, pbar=pbar)
    
    ABABB_mean_result[channel] = spike.apply_linear_filter_withroi(spike_train.spike_train, 
                                                             kernel, ABABB_start, roi=roi, 
                                                             nbins=nbins, pbar=pbar)
    
    scipy.io.savemat(mat_path, {"ABABA": ABABA_mean_result[channel], "ABABB": ABABB_mean_result[channel]})
    #scipy.io.savemat(mat_path, {"ABABA": ABABA_mean_result[channel]})
    # print("processed: "+channel)
pbar.close()
mat_path = os.path.join(_dir, "linear_filter_{setup}.mat")
scipy.io.savemat(mat_path.format(setup='ABABA'), ABABA_mean_result)
scipy.io.savemat(mat_path.format(setup='ABABB'), ABABB_mean_result)

# Visualization

In [ ]:
def ABABB_markers():
    plt.hlines(0, 0,1,linewidth=5,color='k', alpha=0.5)
    plt.hlines(0, 2,3,linewidth=5,color='c', alpha=0.5)
    plt.hlines(0, 4,5,linewidth=5,color='k', alpha=0.5)
    plt.hlines(0, 6,7,linewidth=5,color='c', alpha=0.5)
    plt.hlines(0, 8,9,linewidth=5,color='c', alpha=0.5)
    
def ABABA_markers():
    plt.hlines(0, 0,1,linewidth=5,color='k', alpha=0.5)
    plt.hlines(0, 2,3,linewidth=5,color='c', alpha=0.5)
    plt.hlines(0, 4,5,linewidth=5,color='k', alpha=0.5)
    plt.hlines(0, 6,7,linewidth=5,color='c', alpha=0.5)
    plt.hlines(0, 8,9,linewidth=5,color='k', alpha=0.5)


for channel in [i for i in ABABA_mean_result.keys() if i[0] != '_']:
    demo1 = ABABA_mean_result[channel]
    demo2 = ABABB_mean_result[channel]
    plt.figure(figsize=(24,8))
    spike.plot_curve_with_error_ribbon(demo1, roi, markers=ABABA_markers, label='A')
    plt.title(channel)
    spike.plot_curve_with_error_ribbon(demo2, roi, markers=ABABB_markers, label='B')
    plt.legend()
    plt.savefig(os.path.join(_dir, experiment_id+"_"+channel+".png"), bbox_inches='tight')
    # plt.show()

# Visualiztion alternative

In [ ]:
def alt_visualize(channel):
    demo1 = ABABA_mean_result[channel]
    demo2 = ABABB_mean_result[channel]
    X = np.linspace(roi[0],roi[1],nbins)
    
    def _markers(plotax=plt):
        plotax.hlines(0, 0,1,linewidth=5,color='k', alpha=0.5)
        plotax.hlines(0, 2,3,linewidth=5,color='c', alpha=0.5)
        plotax.hlines(0, 4,5,linewidth=5,color='k', alpha=0.5)
        plotax.hlines(0, 6,7,linewidth=5,color='c', alpha=0.5)
        plotax.hlines(0, 8,9,linewidth=5,color='r', alpha=0.5)

#     def _markers(plotax=plt):
#         plotax.hlines(0, 0,1.5,linewidth=5,color='k', alpha=0.5)
#         plotax.hlines(0, 3,4.5,linewidth=5,color='c', alpha=0.5)
#         plotax.hlines(0, 6,7.5,linewidth=5,color='k', alpha=0.5)
#         plotax.hlines(0, 9,10.5,linewidth=5,color='c', alpha=0.5)
#         plotax.hlines(0, 12,13.5,linewidth=5,color='r', alpha=0.5)

    fig, (ax1,ax2, ax3) = plt.subplots(3,1,figsize=(24,24),sharex=True)
    spike.plot_curve_with_error_ribbon(demo1, roi, label='ABABA', color='#78C2C4', plotax = ax1)
    spike.plot_curve_with_error_ribbon(demo2, roi, label='ABABB', color='#F7C242', plotax = ax1)
    ax1.legend()
    ax1.set_title(channel)

    for each_train in demo1:
        ax2.plot(X, each_train, color='#78C2C4', alpha=0.1, label='')
    ax2.plot(X, np.mean(demo1, axis=0), color='#78C2C4', linewidth=5,label='ABABA')
    ax2.legend()

    for each_train in demo2:
        ax3.plot(X, each_train, color='#B1B479', alpha=0.2, label='')
    ax3.plot(X, np.mean(demo2, axis=0), color='#B1B479', linewidth=5, label='ABABB')
    ax3.legend()

    _markers(ax1)
    _markers(ax2)
    _markers(ax3)

    plt.xlim(roi)
    plt.savefig(os.path.join(_dir, experiment_id+"_"+channel+".png"),bbox_inches='tight')
    # print("generate image for "+channel)

In [ ]:
[alt_visualize(channel) for channel in tqdm(ABABA_mean_result.keys()) if channel[0] != '_'];